<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/110_%E3%82%A4%E3%83%86%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC%E3%81%A8%E3%82%B8%E3%82%A7%E3%83%8D%E3%83%AC%E3%83%BC%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

イテレーターとジェネレーター
============================

イテラブル
----------

`collections.abc` モジュールは、コンテナ、イテラブル、コレクションなどの抽象基底クラスを提供している。あるクラス `C` がどの型であるかは、組み込み関数 `issubclass()` の第2引数に抽象基底クラス（ABC）を渡した `issubclass(C, ABC)` により判定できる。

| 型 | 意味 | ABC | 継承しているクラス | 抽象メソッド | mixin メソッド |
|:--:|:---|:--:|:--:|:---|:---|
| コンテナ | オブジェクトを格納するオブジェクト | Container | | `__contains__`（組み込み演算子 `in` によって呼び出される） | |
| イテラブル | 要素を一度に1つずつ返せるオブジェクト | Iterable | | `__iter__`（組み込み関数 `iter()` によって呼び出される） | |
| コレクション | サイズ付きのイテラブルなコンテナ | Collection | Sized,<br />Iterable,<br />Container | `__contains__`, <br />`__iter__`, <br />`__len__`（組み込み関数 `len()` によって呼び出される） | |
| シーケンス | インデックスで要素を参照できるコレク<br />ション | Sequence | Reversible, <br />Collection | `__len__`, <br />`__getitem__`（添字表記 `x[i]` によって呼び出される） | `__contains__`, `__iter__`, <br />`__reversed__`, `index`, `count` |
| マッピング | キー探索をサポートするコレクション | Mapping | Collection | `__iter__`, <br />`__len__`, <br />`__getitem__`（添字表記 `x[key]` によって呼び出される） | `__contains__`, `keys`, `items`, <br />`values`, `get`, `__eq__`, `__ne__` |
| 辞書ビュー | もとのオブジェクトへの参照をもつイテラ<br />ブル | KeysView, <br/>ValuesView, <br/>ItemsView | MappingView, <br/>Set (ItemsView<br/> は Collection) | | `__contains__`, `__iter__` |

イテラブルは、次のような場面で使われるオブジェクトである。

  * for ループによる反復処理
  * 代入でのアンパック
  * `list()`、`tuple()`、`set()`、`frozenset()`、`dict()` などのコンストラクタの引数として渡される
  * 組み込み関数 `all()`、`any()`、`max()`、`min()`、`sorted()`、`sum()` の引数として渡される

コンテナとイテラブルが提供するインターフェースは重ならない。したがって、イテラブルでないコンテナ（つまり、反復処理ができないコンテナ）や、コンテナでないイテラブル（つまり、オブジェクトを格納しないイテラブル）というのもある。組み込みコンテナや辞書ビューについては、次のとおり。

  * `set` と `frozenset` はコレクションであり、イテラブルとして要素を 1 つずつ返す。とくに、`set` はミュータブルで、`frozenset` はイミュータブルである。
  * `list` と `tuple`、`range` はシーケンスであり、イテラブルとして要素を 1 つずつ返す。とくに、`list` はミュータブルで、それ以外はイミュータブルである。
  * `str` と `bytes` はイミュータブルなシーケンスであり、`str` はイテラブルとして 1 文字ずつ返し、`bytes` はイテラブルとして 1 バイトずつ返す。
  * `dict` はミュータブルなマッピングであり、イテラブルとしてキーを 1 つずつ返す。`in` 演算子はキーに対する結果となる。
  * 辞書ビューは、マッピングオブジェクトの `keys()` メソッド、`values()` メソッド、`items()`  メソッドが返すオブジェクトである。コレクションに似ているが、自身に要素を保持せずに元のオブジェクトへの参照を保っているだけのイテラブルである。`len()` 関数に渡すと要素数が得られる。





In [ ]:
assert list('abc') == ['a', 'b', 'c']
assert list((1, 2, 3)) == [1, 2, 3]
assert list([1, 2, 3]) == [1, 2, 3]  # リストがネストされるわけではない

assert tuple('abc') == ('a', 'b', 'c')
assert tuple((1, 2, 3)) == (1, 2, 3)  # タプルがネストされるわけではない
assert tuple([1, 2, 3]) == (1, 2, 3)

assert set('abc') == {'a', 'b', 'c'}  # 要素の並び順序は無視
assert set([1, 2, 3]) == {1, 2, 3}  # 要素の並び順序は無視

assert dict([('foo', 100), ('bar', 200)]) == dict(foo=100, bar=200)
assert list(dict(foo=100, bar=200)) == ['foo', 'bar']

assert all(range(10)) == False  # 全ての要素が真ならば (もしくは空ならば) True
assert any(range(10)) == True  # いずれかの要素が真ならば True
assert max(range(10)) == 9  # 最大の要素
assert min(range(10)) == 0  # 最小の要素
assert sum(range(10)) == 45  # 要素を左から右へ合計

it = tuple('Python')  # ('P', 'y', 't', 'h', 'o', 'n')
assert sorted(it) == ['P', 'h', 'n', 'o', 't', 'y']  # イテラブルの要素を並べ替えた新たなリストを返す
assert sorted(it, key=str.lower) == ['h', 'n', 'o', 'P', 't', 'y']  # key は1つの引数を受け取る呼び出し可能オブジェクトで、要素を変換する
assert sorted(it, reverse=True) == ['y', 't', 'o', 'n', 'h', 'P']  # reverse が True の場合、比較が反転したものとして並び替えられる

イテレーター
------------

**イテラブル**（**反復可能オブジェクト**ともいう）の厳密な定義は、「`__iter__()` メソッドを持ち、その `__iter__()` メソッドがイテレーターを返すようなオブジェクト」である。

**イテレーター**（iterator）とは、イテレータープロトコル（iterator protocol）をサポートするオブジェクトのことをいう。イテレータープロトコルは、次の三つの規則からなる。

  1. `__next__()` メソッドは、次の要素を返す。
  2. `__next__()` メソッドは、次の要素がないとき、`StopIteration` 例外を送出する。
  3. `__iter__()` メソッドは、自身を返す。

イテレータープロトコルは、要素を列挙する手段を独立に用意するというイテレーターパターン（デザインパターンの 1 つ）を Python の言語仕様に組み込んだものである。

`StopIteration` 例外オブジェクトは `value` 属性を持つ。 `value` 属性の値は、インスタンス化の際にコンストラクタ引数として与えられ、デフォルトは `None` である。

次の例は、`Count3Do` が `limit` までの 3 の倍数と 3 が付く自然数を返すイテレーターのクラスで、 `Gain3Do` がイテラブルのクラスである。

In [ ]:
from collections.abc import Iterable, Iterator


class Count3Do:
    """3の倍数と3が付く数を数え上げる"""

    def __init__(self, limit=100):
        self.limit = limit
        self.counter = 0

    def __next__(self):
        while True:
            self.counter += 1
            if self.counter > self.limit:
                raise StopIteration()
            if self.counter % 3 == 0:
                break
            elif "3" in str(self.counter):
                break
        return self.counter

    def __iter__(self):
        return self


class Gain3Do:
    """3の倍数と3が付く数を得る"""

    def __init__(self, limit=100):
        self.limit = limit

    def __iter__(self):
        return Count3Do(self.limit)


assert issubclass(Count3Do, Iterator)
assert issubclass(Gain3Do, Iterable)

it = Gain3Do(50)
for n in it:
    print(n, end=" ")

3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 

Python のインタープリターは、for 文に対して、次の一連の操作を自動的に行う：

  1. `in` の右にある式を一度だけ評価し、評価結果としてイテラブルを得ると、`_iter__()` メソッドを呼び出してイテレーターを取得する。
  2. イテレーターの `__next__()` メソッドを呼び出して次の要素を取得し、`in` の左にある変数に代入する。for 文の本体を実行する。
  3. 2を繰り返す。StopIteration 例外を捕捉したら、ループを停止し、else 節があれば実行する。StopIteration 例外を捕捉するまえに `break` を実行したときには、ループを中断し、else 節を実行しない。

組み込み関数 `iter()` はイテラブルを引数に取り実行するとき、`__iter__()` メソッドを呼び出すことにより、イテレーターを取得する。

組み込み関数 `next()` はイテレーターを引数に取り実行するとき、`__next__()` メソッドを呼び出すことにより、次の要素を取得する。`next()` は第 2 引数を取ることができ、StopIteration 例外を捕捉した場合に第 2 引数の値を返す（StopIteration 例外は送出されない）。

したがって、上記コードの for 文は、次の while 文と等価である：

In [ ]:
it = Gain3Do(50)
n_iter = iter(it)
while True:
    try:
        print(next(n_iter), end=" ")
    except StopIteration:
        del n_iter
        break

3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 

ここで、二つの疑問が浮かぶ。

一つ目の疑問。`Count3Do` クラスの `__iter__()` メソッドは何の役にも立っておらず、不要ではないか？ 確かに、`Count3Do` クラスの定義から `__iter__()` メソッドの記述をコメントアウトしても、for 文は問題なく動作する。こうするとイテレータープロトコル違反なので、当然 `issubclass(Count3Do, Iterator)` は `False` を返す。

二つ目の疑問。イテレーター自身が `__iter__()` メソッドを提供しなければならないのなら、それ自身をイテラブルとして使えば済むのではないか？ つまり、`Count3Do` クラスだけ用意すれば、`Gain3Do` クラスは不要ではないか？ 確かに、`Count3Do` クラスのインスタンスに対して、for 文を実行しても、同じ結果を得ることができる。しかしながら、`Count3Do` クラスのインスタンスにもう一度 for 文を実行すると、何も得られない。これは、一度目の for 文の時点で、イテレーターは尽きており、それ以降は `__next__()` を何度呼んでも StopIteration 例外を送出するからである。`Gain3Do` クラスのインスタンスの `__iter__()` メソッドは毎回イテレーターオブジェクトを生成して返すから、このような問題は起こらない。


In [ ]:
# Count3Do をイテラブルとして使う
it = Count3Do(50)
for n in it:
    print(n, end=" ")
else:
    print("\n--------")
# もう一度 for 文を実行
for n in it:
    print(n, end=" ")  # 実際には実行されない
try:
    print(next(it))  # next() で次の要素を取得しようとすると？
except StopIteration:
    print(f"上限 {it.limit} に達しました。もう返すものがありません")

3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 
--------
上限 50 に達しました。もう返すものがありません


このように、**イテレーターは使い捨てオブジェクトというべきものである**。このことは、コンテナに直接イテレータープロトコルをサポートさせる場合に問題を起こす。データを格納するコンテナが一度しか反復処理できないというのは、不便である。コレクションを実装するなら、ちゃんとイテレータークラスを分けて実装せよ、ということである。`list` などの組み込みコレクションは、そのように実装されている。

一方、コンテナではないイテラブルは、必要な場面で、そのたびごとに生成すればよいであろう。反復処理をさせるためだけのオブジェクトを実装するケースでは、イテレータークラスとイテラブルクラスを分けて実装するのはむしろ冗長に思える。3 の倍数と 3 が付く数を数え上げる例がまさにそのような場合である。

ジェネレーター
--------------

Python はクラスを使用せずにイテレーターを簡潔に記述する仕組みを用意している。それがジェネレーターである。

**ジェネレーター**（generator）は、イテレーターを返す関数である。通常の関数に似ているが、 yield 式または yield 文を持つ点で異なる。ジェネレーターが返すイテレーターを**ジェネレーターオブジェクト**または**ジェネレーターイテレーター**と呼ぶ。ジェネレーターオブジェクトを簡単にジェネレーターと呼ぶこともある（区別のため関数のほうを**ジェネレーター関数**と呼ぶ）。

ジェネレーターオブジェクトは、イテレーターとして機能するために以下のような性質を持つ。

  * 実行状態を持つ。ここで実行状態とは、実行位置やローカル変数、未処理の try 文などである。
  * yield 式のたびに実行状態を記憶し、処理を一時的に中断する。
  * `__next__()` メソッドによりジェネレーターオブジェクトが再開されると、中断した位置を取得する──通常の関数が実行のたびに新しい状態から開始するのと対照的である。
  * `__next__()` メソッドは yield 式の評価結果を返す。関数の最後または return 文に到達した場合には `StopIteration` 例外を送出する。return 文に書いた式の値は、 `StopIteration` の `value` 属性に格納される。

ジェネレーターの呼び出しは、return 文の結果が返されるのではなく、ジェネレーターオブジェクトが返されることに注意する。

In [ ]:
def my_gen():
    print("#" * 10)
    yield "ホップ"
    print("-" * 10)
    yield "ステップ"
    print("=" * 10)
    return "何か書いとけ"
    yield "ジャンプ"

my_gen()

<generator object my_gen at 0x7dddbb0e9d20>

`my_gen()` を呼び出すと、ジェネレーターオブジェクトが返されることがわかる。 `__next__()` メソッドと `__iter__()` メソッドは自動的に定義される:

In [ ]:
generator = my_gen()
assert hasattr(generator, '__next__')
assert hasattr(generator, '__iter__')
assert generator.__iter__() == generator

`__next__()` メソッドの振る舞いを確認するため、このジェネレーターオブジェクトに対して `next()` 関数を実行してみる:

In [ ]:
print(f"1: {next(generator)}")
print(f"2: {next(generator)}")
try:
    print(f"3: {next(generator)}")
except StopIteration as err:
    print(f"{type(err).__name__}： {err.value=}")

##########
1: ホップ
----------
2: ステップ
StopIteration： err.value='何か書いとけ'


最初に `next(generator)` を呼び出すと、`my_gen()` の関数定義を最初の yield 文に到達するまで実行し、文字列 "ホップ" が返される。再び `next(generator)` を呼び出すと、yield 文から再開し二つ目の yield 文に到達して文字列 "ステップ" が返される。再び `next(generator)` を呼び出すと、二つ目の yield 文から再開し return 文の箇所でその値を引数として `StopIteration` 例外が送出される。このように、yield 式または yield 文に到達することなくジェネレーターが終了したときには、`StopIteration` 例外が送出される。このあと `next(generator)` を呼び出しても return 文から再開するから直ちに StopIteration 例外が送出される。

3 の倍数と 3 が付く数を数え上げるジェネレーターは、次のように書ける。

In [ ]:
def Count3DoGen(limit=100):
    """3の倍数と3が付く数を数え上げる"""
    counter = 0
    while True:
        counter += 1
        if counter > limit:
            return
        if counter % 3 == 0:
            yield counter
        elif "3" in str(counter):
            yield counter


for n in Count3DoGen(50):
    print(n, end=" ")

3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 

### 遅延評価 ###


ジェネレーターオブジェクトはイテレーターなので、使い捨てとなる。そもそも、ジェネレーターを使う場面は、コンテナを必要とせず反復処理だけ行いたい場合なので、使い捨てでも問題ない。逆に、このような場面でイテラブルなコンテナを使うと、コンテナを生成する分、メモリを多く消費することになって効率が悪い。

たとえば、3 の倍数と 3 が付く数を数え上げる処理は、リストを生成する形で次のようにも書ける。

In [ ]:
def find_3do(limit=100):
    n = 1
    ret = []
    while n <= limit:
        if n % 3 == 0:
            ret.append(n)
        elif "3" in str(n):
            ret.append(n)
        n += 1
    return ret


ns = find_3do(50)  # サイズ25のリストを生成する
print(ns)

[3, 6, 9, 12, 13, 15, 18, 21, 23, 24, 27, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 45, 48]


`sys.getsizeof()` 関数を使用して、引数 `limit` をデフォルト（100）で、`Count3DoGen()` 関数が生成するジェネレータオブジェクトと、`find_3do()` 関数が生成するリストのメモリ消費量を比較すると、後者は前者の4倍以上もある。`limit` を増やす場合、`Count3DoGen()` 関数が生成するジェネレータオブジェクトのメモリ消費量は増えないが、`find_3do()` 関数が生成するリストはサイズが増えてメモリ消費量も増える。

In [ ]:
import sys

print(f"{sys.getsizeof(Count3DoGen())=}bytes")
print(f"{sys.getsizeof(find_3do())=}bytes")

print(f"{sys.getsizeof(Count3DoGen(1000))=}bytes")
print(f"{sys.getsizeof(find_3do(1000))=}bytes")

sys.getsizeof(Count3DoGen())=104bytes
sys.getsizeof(find_3do())=472bytes
sys.getsizeof(Count3DoGen(1000))=104bytes
sys.getsizeof(find_3do(1000))=4216bytes


メモリ消費量の違いは、ジェネレーターが必要なときだけ値を計算するために生じる。このように、値が必要になるまで計算しないことを**遅延評価**（lazy evaluation）という。遅延評価は、必要のない値を計算する無駄を省略するという利点もある。他方、`len()` 関数を使用して全体の要素数を知ることはできない。生成される要素のサイズを知りたい場合は、`list()` でリストオブジェクトに変換しておく必要がある。

### ジェネレーターオブジェクトのメソッド ###

ジェネレーターオブジェクトは、普通のイテレーターより機能が拡張され、以下のメソッドも持つ。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `send(value)` | ジェネレーター関数の内部へ値 `value` を「送り」、実行を再開する。引数の `value` はその時点の yield 式の結果になる。`send()` メソッドは次にジェネレーターが生成<br />した値を返し、ジェネレーターが次の値を生成することなく終了すると `StopIteration` を送出する | 次の値 |
| `throw(value)` | `value` に例外インスタンスを指定すると、ジェネレーターが中断した位置でその例外を発生させる。このとき、発生させた例外をジェネレーター関数内部で処理して<br />実行を続けると、yield 式によって次の値が呼び出した側に返される。yield 式に到達しなければ、`StopIteration` 例外が発生する。発生させた例外をジェネレーター<br />関数内部で処理しない場合、もしくは違う例外を発生させるなら、その例外は呼び出し元へ伝搬される | 次の値 |
| `close()` | ジェネレーターが中断した位置で、`GeneratorExit` 例外を発生させる。この例外は呼び出し元に伝播しない。ジェネレーターが普通に、または例外により既に終了して<br />いる場合、`close()` は何も行わない | `None` |

`send()` メソッドは、ジェネレーターを再開するという点では `__next__()` メソッドと同じであるが、先に再開位置にある yield 式に「値を送る」。

yield 文ではなく yield 式 `(yield <expr>)` を使うと、expr の値を yield すると同時に、yield 式自体の評価結果として値が得られる。`__next__()` メソッドを呼ぶときは、yield 式の値として `None` が得られる。一方、`send(value)` を呼ぶときは、yield 式の値として `value` が得られる。ただし、最初に（`__next__()` メソッドではなく）いきなり`send()` メソッドを呼び出してジェネレーターを開始する場合は、値を受け取る yield 式が存在しないので、 `None` を引数として呼び出さなければならない。

なお、yield 演算子の優先順位は低いので yield 式の前後に必ず括弧を付けるようにするのが無難。

以下のコードにあるのは 1 ずつ増える単純なカウンターであるが、`val = (yield i)` として yield 式の値を変数 `val` に保存し利用している。

In [ ]:
def counter(maximum):
    i = 1
    while i <= maximum:
        val = (yield i)
        if val is not None:
            i = val
        else:
            i += 1

def main():
    generator = counter(3)  # 3 までカウント
    print(f"1: {next(generator)=}")
    print(f"2: {next(generator)=}")
    print(f"3: {generator.send(0)=}")  # カウントを 0 に設定
    print(f"4: {next(generator)=}")
    try:
        print(f"5: {generator.send(4)=}")  # カウントを 4 に設定
    except StopIteration:
        print("カウントできません")

if __name__ == "__main__":
    main()

1: next(generator)=1
2: next(generator)=2
3: generator.send(0)=0
4: next(generator)=1
カウントできません


`main()` 関数の 4 行目以降の処理は以下のようになる:

  1. `print(f"3: {generator.send(0)=}")`:  
`generator.send(0)` を呼び出す。ジェネレーターは、`val = (yield i)` の行から再開するが、再開する前に `(yield i)` の評価として `0` が得られ `val` に代入される。再開して if 文で `i = val` が実行され、カウンタ変数 `i` は `0` となる。while ループを続けて `val = (yield i)` の行に達すると、`i` の値 `0` が yield されて中断する。`main()` 関数に戻り、標準出力に印字する。
  2. `print(f"4: {next(generator)=}")`:  
`next(generator)` を呼び出す。ジェネレーターは、`val = (yield i)` の行から再開するが、再開する前に `(yield i)` の評価として `None` が得られ `val` に代入される。再開して else 節で `i += 1` が実行され、カウンタ変数 `i` は `1` となる。while ループを続けて `val = (yield i)` の行に達すると、`i` の値 `1` が yield されて中断する。`main()` 関数に戻り、標準出力に印字する。
  3. `print(f"5: {generator.send(4)=}")`:  
`generator.send(4)` を呼び出す。ジェネレーターは、`val = (yield i)` の行から再開するが、再開する前に `(yield i)` の評価として `4` が得られ `val` に代入される。再開して if 文で `i = val` が実行され、カウンタ変数 `i` は `4` となる。while ループが条件により終了する。ジェネレーターが終了し、`StopIteration` 例外が送出される。`main()` 関数に戻り、例外処理が行われる。

`counter()` は内部で例外処理を行わないので、 `throw()` メソッドで例外を投げると、その例外がそのまま呼び出し元へ伝播される。以下は `throw()` メソッドを使用してカウンターの処理を終了させる例である:

In [ ]:
generator = counter(100)  # 100 までカウント
try:
    for n in generator:
        print(n)
        if n > 2:
            generator.throw(ValueError("Invalid Value"))
except ValueError as err:
    print(err)

1
2
3
Invalid Value


`close()` メソッドは `GeneratorExit` 例外を発生させるが、`GeneratorExit` 例外は一般的な例外階層には属していないことに注意する。つまり、`GeneratorExit` 例外は一般的な `Exception` 例外では捕捉できない。以下のコードのように、finally 節に後始末（リソースの解放など）をするコードを記述しておくと、`close()` でもそのコードが確実に実行される。

In [ ]:
def echo(value=None):
    try:
        while True:
            try:
                value = (yield value)
            except Exception as e:
                value = e
    finally:
        print("終了")

generator = echo(1)
assert next(generator) == 1
# next() 関数で呼び出すと yield 式の値は None となる
assert next(generator) is None
# send() メソッドで yield 式の値を送る
assert generator.send(2) == 2
# 例外を投げる
e = generator.throw(TypeError, "spam")
assert e.__class__.__name__ == "TypeError"
assert str(e) == "spam"
# echo の内部で例外が処理されるのでジェネレーターは終了しない
assert generator.send(3) == 3
# close() メソッドで finally 節が実行される（リソース解放などの後始末が可能）
generator.close()

終了


`close()` メソッドは、ジェネレーターが開始され、終了する前に削除された場合に、自動的に呼び出される。

In [ ]:
generator = echo(1)
next(generator)
del generator

終了


### サブジェネレーター ###

yield 式には、もう一つの形式がある。それは次のような構文である。

``` python
yield from <expr>
```

expr は評価結果がイテラブルとなる式でなければならない。そのイテラブルを反復処理することで生成された値は呼び出し元に返される。yield from 式で使用するイテラブルがジェネレーターの場合、このジェネレーターを**サブジェネレーター**と呼ぶ。yield from 式でサブジェネレーターを利用することを「**サブジェネレーターへの委譲**」と表現する。委譲されたサブジェネレーターは、さらに yield from 式で委譲できる。こうしてジェネレーターの処理が連鎖することから、サブジェネレーターへの委譲が行われることを「**ジェネレーターの鎖（チェーン）**」と表現する。

サブジェネレーターへの委譲は、サブジェネレーターが全ての値を取り尽くすまで完了しないことに注意する。つまり、ジェネレーターはサブジェネレーターへの委譲のところで中断し、サブジェネレーターへの委譲が完了した後、残りの処理が再開する。

簡単な例を以下に示す。

In [ ]:
def sample_geniter():
    yield from range(4)
    print("sample_geniter: 終了")

for n in sample_geniter():
    print(n)

print("スクリプト終了")

0
1
2
3
sample_geniter: 終了
スクリプト終了


`send()` メソッドを使用する場合、送った値はサブジェネレーターに渡される。サブジェネレーターが yield した値が `send()` メソッドの戻り値となる。

In [ ]:
def sub_gen():
    i = 1
    while True:
        x = (yield i)
        i += x

def delegate_gen():
    yield from sub_gen()
    print("終了")  # sub_gen は永遠に値を生成し続けるので、この行が実行されることはない

generator = delegate_gen()
assert generator.send(None) == 1  # 開始時は None を渡す
assert generator.send(10) == 11
assert generator.send(10) == 21

ジェネレーター式
----------------

リスト内包表記と同じ記法を丸括弧で囲んだものは**ジェネレーター式**（generator expression）と呼ばれ、評価するとジェネレーターオブジェクトが得られる（tuple が得られるわけではないことに注意する）。また、リスト内包表記と同様に、ジェネレーター式が作るブロックでは独自のスコープが形成される。

In [ ]:
x = "some string"
gen = (x for x in range(50) if x % 3 == 0 or "3" in str(x))
for n in gen:
    print(n, end=" ")
assert x == "some string"

0 3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 

ジェネレーターオブジェクトはイテラブルであるから、イテラブルを要求する関数の引数としてジェネレーター式を渡すことができる。関数の唯一の引数として渡す場合には、丸括弧を省略できる。

In [ ]:
tuple(x for x in range(10))

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [ ]:
dict((i, c) for i, c in enumerate(["a", "b", "c", "d", "e"]))

{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e'}

イテレーターを生成する組み込み関数
----------------------------------

``` python
iter(object, sentinel)
```

`iter()` 関数は第 2 引数を取ることもできる。第 2 引数があるかどうかによって第 1 引数の解釈は大きく異なる。第 2 引数 `sentinel` が与えられた場合、第 1 引数は呼び出し可能オブジェクトでなければならない。この場合に `iter()` が返すイテレーターは、 `__next__()` メソッドを呼び出すごとに引数なしで第 1 引数を呼び出す。その戻り値が `sentinel` と等しければ、 StopIteration 例外が送出される。それ以外の場合は戻り値がそのまま返される。

In [ ]:
class Simple3Do:
    """3の倍数と3が付く数を数え上げる（非イテレーター版）"""

    def __init__(self, limit=100):
        self.limit = limit
        self.counter = 0

    def __call__(self):
        while True:
            self.counter += 1
            if self.counter > self.limit:
                return
            if self.counter % 3 == 0:
                break
            elif "3" in str(self.counter):
                break
        return self.counter


if __name__ == "__main__":
    simple3do = Simple3Do(50)
    for n in iter(simple3do, None):
        print(n, end=" ")

3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 

``` python
enumerate(iterable, start=0)
```

この関数によって返されたイテレーターの `__next__()` メソッドは、（デフォルトでは 0 となる `start` からの）カウントと、`iterable` 上のイテレーションによって得られた値を含むタプルを返す。

``` python
zip(iterable[, iterable[, iterable[...]]], /, *, strict=False)
```

この関数は、複数のイテラブルからそれぞれ一つずつ取った要素からなるタプルを生成（yield）するようなイテレーターを返す。デフォルトでは、最も短いイテラブルが使い尽くされた段階で止まる。`strict=True` オプションを指定する場合は、あるイテラブルが他のイテラブルよりも先に使い尽くされたときに `ValueError` 例外を送出する。

複数のイテラブルを行に見立てれば、`zip()` は行を列に、また列を行に変換する。これは行列の転置とよく似ている。

In [ ]:
matrix = [
    (1, 2, 3),
    (4, 5, 6),
    (7, 8, 9),
]
t_matrix = [col for col in zip(*matrix, strict=True)]
for x in t_matrix:
    print(x)

(1, 4, 7)
(2, 5, 8)
(3, 6, 9)


``` python
reversed(seq)
```

この関数は、シーケンスの要素を逆順に取り出すイテレーターを返す。順番は単なる逆順であって、降順で整列した結果の順番ではないことに注意する。なお、リストなら `reverse()` メソッドを使ってインプレースに逆転させることができる。`reversed()` 関数はタプルなどリスト以外のシーケンスや、辞書ビューにも適用できる。辞書 `d` について `reversed(d)` は `reversed(d.keys())` へのショートカットである。

In [ ]:
li = list("いろはにほへと")

print(f"{reversed(li)=}")  # reversed() 関数の戻り値は list 型専用のイテレーター
assert list(reversed(li)) == ["と", "へ", "ほ", "に", "は", "ろ", "い"]  # ただの逆順

print(f"{li.reverse()=}")  # reverse() メソッドはインプレースで並べ替えるので、戻り値は None
assert li == ["と", "へ", "ほ", "に", "は", "ろ", "い"]  # ただの逆順

# sorted() 関数で reverse=True を指定した場合は、降順で整列した結果を返す
assert sorted(li, reverse=True) == ["ろ", "ほ", "へ", "は", "に", "と", "い"]

reversed(li)=<list_reverseiterator object at 0x7dddbaf31600>
li.reverse()=None


``` python
map(callable, iterable[, iterable[, iterable[...]]])
```

`map(callable, iterable)` を実行すると、`iterable` が返す要素を引数として渡して `callable` を呼び出し、その結果を生成（yield）するようなイテレーターを返す。追加の `iterable` 引数が渡された場合、`callable` は渡されたイテラブルと同じ数の引数を取らなければならない。複数のイテラブルが渡された場合、そのうちで最も短いイテラブルが使い尽くされた段階で止まる。

In [ ]:
gen = map(lambda x, y: x + y, [1, 2, 3, 4], [4, 3, 2])
for n in gen:
    print(n, end=" ")

5 5 5 

``` python
filter(callable or None, iterable)
```

`filter(callable, iterable)` を実行すると、`iterable` が返す要素を引数として渡して `callable` を呼び出し、`callable` が真を返す要素だけを生成（yield）するようなイテレーターを構築する。これはジェネレーター式 `(item for item in iterable if callable(item))` と同等である。

In [ ]:
it = filter(lambda x: True if x % 3 == 0 or "3" in str(x) else False, range(50))
for n in it:
    print(n, end=" ")

0 3 6 9 12 13 15 18 21 23 24 27 30 31 32 33 34 35 36 37 38 39 42 43 45 48 

`filter(None, iterable)` は `filter(lambda x: x, iterable)` と等価であり、`iterable` のうち偽であるものがすべて取り除かれる。これはジェネレーター式 `(item for item in iterable if item)` と同等である。

In [ ]:
li = [1, 0, 2, 0, 3, 0, 4, 0, 5, 0]
assert list(filter(None, li)) == list(x for x in li if x) == [1, 2, 3, 4, 5]

イテレーターを構築する組み込み関数は、すべて遅延評価である。for 文に渡されたり、`list()`、`tuple()`、`set()`、`frozenset()`、`dict()` などのコンストラクタでラップされたりするなどして反復処理されるまで、要素が実際に処理されることはない。

itertools
---------

標準ライブラリの itertools モジュールは、イテレーターを生成する関数を提供している。

### chain ###

``` python
itertools.chain(iterable1, iterable2, ...)
```

複数のイテラブルを連結したイテレーターを返す。

In [ ]:
from itertools import chain
it = chain('ABC', 'DEF')
assert list(it) == ['A', 'B', 'C', 'D', 'E', 'F']

### islice ###

``` python
itertools.islice(iterable, stop)
itertools.islice(iterable, start, stop[, step])
```

イテラブルの、指定した範囲の要素を生成（yield）するようなイテレーターを返す。イテラブル以外の引数の意味は `range()` の引数とほとんど同じ。`stop` が `None` の場合、最後の要素まで yield する。`start` が未指定または `None` の場合、最初の要素から yield する。`step` が未指定または `None` の場合、ステップはデフォルトの 1 になる。

In [ ]:
from itertools import islice
it = islice('ABCDEFG', 2)
assert list(it) == ['A', 'B']
it = islice('ABCDEFG', 2, None)
assert list(it) == ['C', 'D', 'E', 'F', 'G']

### zip_longest ###

``` python
itertools.zip_longest(iterable[, iterable[, iterable[...]]], /, *, fillvalue=None)
```

組み込み関数 `zip()` とほとんど同じであるが、`iterable` の長さが違う場合、最も長いイテラブルが使い尽くされるまでタプルを yield する。足りない値は `fillvalue` で埋められる。

In [ ]:
from itertools import zip_longest
it = zip_longest('ABCD', 'xy', fillvalue='-')
assert list(it) == [('A', 'x'), ('B', 'y'), ('C', '-'), ('D', '-')]

### takewhile ###

``` python
itertools.takewhile(callable, iterable)
```

`iterable` が返す要素を引数として渡して `callable` を呼び出し、`callable` が真を返すうちは要素を yield するが、`callable` が偽を返したら直ちに停止するようなイテレーターを返す。

In [ ]:
from itertools import takewhile
it = takewhile(lambda x: x < 5, [1, 4, 6, 4, 1])
assert list(it) == [1, 4]

### product ###

``` python
itertools.product(iterable1, iterable2, ..., /, *, repeat=1)
```

イテラブルたちをそれぞれ返される要素の集合とみるときの、直積集合の要素となるタプルを yield するようなイテレーターを返す。イテラブル自身との直積を計算するためには、オプションの `repeat` キーワード専用引数に繰り返し回数を指定する。

In [ ]:
from itertools import product
it = product("ABC", repeat=2)
print(list(it))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'B'), ('B', 'C'), ('C', 'A'), ('C', 'B'), ('C', 'C')]


### permutations ###

``` python
itertools.permutations(iterable, r=None)
```

`iterable` の要素からなる長さ `r` の順列となるタプルを連続的に yield するようなイテレーターを返す。`r` が `None` の場合、`iterable` の長さの順列となる。

In [ ]:
from itertools import permutations
it = permutations("ABC")
print(list(it))

[('A', 'B', 'C'), ('A', 'C', 'B'), ('B', 'A', 'C'), ('B', 'C', 'A'), ('C', 'A', 'B'), ('C', 'B', 'A')]


### combinations ###

``` python
itertools.combinations(iterable, r)
```

`iterable` の要素からなる長さ `r` の組合せとなるタプルを yield するようなイテレーターを返す。

In [ ]:
from itertools import combinations
it = combinations("ABC", 2)
print(list(it))

[('A', 'B'), ('A', 'C'), ('B', 'C')]


### combinations_with_replacement ###

``` python
itertools.combinations_with_replacement(iterable, r)
```

`iterable` の要素からなる長さ `r` の重複組合せとなるタプルを yield するようなイテレーターを返す。

In [ ]:
from itertools import combinations_with_replacement
it = combinations_with_replacement("ABC", 2)
print(list(it))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]


### groupby ###

``` python
itertools.groupby(iterable, key=None)
```

`key` に 1 つの引数を取る関数を指定した場合、その関数に `iterable` の要素を渡したときの戻り値 `key_value` と、関数の戻り値が同じ `key_value` となる `iterable` の中のグループ `group` からなるタプル `(key_value, group)` を yield するようなイテレーターを返す。ここではグループは、`iterable` の中の連続する要素を yield するようなイテレーターのことを指している。

`itertools.groupby()` が生成するイテレーターは、`key` 関数の戻り値が変わるたびに新しい `(key_value, group)` を yield する。このため、そのイテレーターの挙動は `iterable` の中の順番に依存する。 `iterable` の中の順番に関係なく `key` 値で 1 つのグループを生成したい場合は、同じ `key` 関数で `iterable` をソートしておく必要がある。

次のコードは、タプルのリスト `animals` に関し、各タプルの第 2 要素を返す無名関数を `key` に渡して `itertools.groupby()` を呼び出している。 `animals` をそのまま使う `itertools.groupby()` が生成するイテレーターの挙動と、同じ無名関数で `animals` をソートしたリストを使う `itertools.groupby()` が生成するイテレーターの挙動の違いに注意する。

In [ ]:
from itertools import groupby

animals = [("チーター", "哺乳類"),
           ("イヌ", "哺乳類"),
           ("コウモリ", "哺乳類"),
           ("ダチョウ", "鳥類"),
           ("ツバメ", "鳥類"),
           ("クジラ", "哺乳類"),
           ("ペンギン", "鳥類"),
           ("ダチョウ", "鳥類")]

for key_value, group in groupby(animals, lambda x: x[1]):
    print(f"{key_value}: {'・'.join(x[0] for x in group)}")

print("-" * 42)

sorted_animals = sorted(animals, key=lambda x: x[1])
for key_value, group in groupby(sorted_animals, lambda x: x[1]):
    print(f"{key_value}: {'・'.join(x[0] for x in group)}")

哺乳類: チーター・イヌ・コウモリ
鳥類: ダチョウ・ツバメ
哺乳類: クジラ
鳥類: ペンギン・ダチョウ
------------------------------------------
哺乳類: チーター・イヌ・コウモリ・クジラ
鳥類: ダチョウ・ツバメ・ペンギン・ダチョウ


`key` が未指定または `None` の場合、`key` 関数のデフォルトは恒等関数になり要素をそのまま返す。この場合、`itertools.groupby()` が生成するイテレーターは、要素とその要素のグループからなるタプル を yield する。`iterable` がソートされていないと、同じ要素でも同じグループに入らないことに注意する。

In [ ]:
from itertools import groupby

for key_value, group in groupby('AAAABBBCCDAABBB'):
    print(f"{key_value}: {''.join(group)}")

A: AAAA
B: BBB
C: CC
D: D
A: AA
B: BBB


要素を共有するイテレーターを返すイテレーターというのは、なにかトリッキーなことをしているように思えるが、イテレータークラスに内部メソッドとしてジェネレーターを定義し、`__next__()` メソッドでそのジェネレーターを返すようにすれば、等価な Python コードを書くことができる。以下のコードは、[公式ドキュメント](https://docs.python.org/ja/3/library/itertools.html#itertools.groupby)のコードを参考にしてイテレータークラス `GroupBy` を書いた。

In [ ]:
class GroupBy:
    def __init__(self, iterable, key=None):
        if key is None:
            key = lambda x: x  # noqa: E731
        self.keyfunc = key
        self.it = iter(iterable)
        self.tgtkey = self.currkey = self.currvalue = object()

    def __iter__(self):
        return self

    def __next__(self):
        self.id = object()
        while self.currkey == self.tgtkey:
            self.currvalue = next(self.it)  # Exit on StopIteration
            self.currkey = self.keyfunc(self.currvalue)
        self.tgtkey = self.currkey
        return (self.currkey, self._grouper(self.tgtkey, self.id))

    def _grouper(self, tgtkey, id):
        while self.id is id and self.currkey == tgtkey:
            yield self.currvalue
            try:
                self.currvalue = next(self.it)
            except StopIteration:
                return
            self.currkey = self.keyfunc(self.currvalue)


for key_value, group in GroupBy("AAAABBBCCDAABBB"):
    print(f"{key_value}: {''.join(group)}")

A: AAAA
B: BBB
C: CC
D: D
A: AA
B: BBB


### batched ###

``` python
itertools.batched(iterable, n)
```

`iterable` から得られるデータを `n` 個ごとに 1 つのタプルにまとめる。一番最後のバッチは `n` 個より少なくなる可能性がある。Python 3.12 で追加。

``` python
from itertools import batched
for batch in batched('ABCDEFG', 3):
    print(batch)
```

このコードを実行すると、次のように出力する。

``` text
('A', 'B', 'C')
('D', 'E', 'F')
('G',)
```

operator
--------

標準ライブラリの `operator` モジュールは

  * `map()`、`filter()`、`itertools.takewhile()` などの `callable` 引数
  * `itertools.groupby()`、`sorted()` などの `key` 引数

などに渡せる「呼び出し可能なオブジェクトを返す関数」を提供する。

``` python
operator.itemgetter(item)
operator.itemgetter(*items)
```

`operator.itemgetter(item)` が返す関数オブジェクトは、シーケンスやマッピング `obj` に対して、`__getitem__()` メソッドを使って添字表記 `obj[item]` に相当する処理を実行できる。たとえば、`f = itemgetter(2)` とした後で、`f(obj)` を呼び出すと `obj[2]` を返す。続けて書くと `itemgetter(2)(obj) == obj[2]` である。引数を `item1, item2, ...` と複数指定する場合は、`(obj[item1], obj[item2], ...)` を返すような関数である。次と等価である:

``` python
def itemgetter(*items):
    if len(items) == 1:
        item = items[0]
        def g(obj):
            return obj[item]
    else:
        def g(obj):
            return tuple(obj[item] for item in items)
    return g
```

In [ ]:
from operator import itemgetter

mylist = [(3, 3, 1), (4, 2, 1), (2, 1, 3), (1, 1, 4)]
assert sorted(mylist) == [(1, 1, 4), (2, 1, 3), (3, 3, 1), (4, 2, 1)]  # 第1要素でソート
assert sorted(mylist, key=itemgetter(1, 2)) == [(2, 1, 3), (1, 1, 4), (4, 2, 1), (3, 3, 1)]  # 第2要素と第3要素でソート

In [ ]:
from itertools import groupby
from operator import itemgetter

received_mails = [
    {"date": "2022-01-02", "address": "alice@example.com", "subject": "subject1"},
    {"date": "2022-02-03", "address": "bob@example.com", "subject": "subject2"},
    {"date": "2022-03-04", "address": "chris@example.com", "subject": "subject3"},
    {"date": "2022-04-05", "address": "alice@example.com", "subject": "subject4"},
    {"date": "2022-05-06", "address": "bob@example.com", "subject": "subject5"},
]
sorted_received_mails = sorted(received_mails, key=itemgetter("address"))
for address, mails in groupby(sorted_received_mails, itemgetter("address")):
    print(address, ",".join(mail["subject"] for mail in mails))

alice@example.com subject1,subject4
bob@example.com subject2,subject5
chris@example.com subject3


`operator.itemgetter()`はスライスにも対応している。スライスは組み込み型である `slice` 型のインスタンスである。

``` python
slice(stop)
slice(start, stop, step=None)
```

`range(start, stop, step)` で指定されたインデックスのセットを表すスライスオブジェクトを返す。引数 `start` と `step` のデフォルトは `None`。

スライスオブジェクトには読み出し専用のデータ属性 `start`, `stop`, `step` があり、これらは単に引数の値（またはそのデフォルト値）を返す。 Python 3.12 からは、スライスオブジェクトがハッシュ可能となった（ただし、属性 `start`, `stop`, `step` がすべてハッシュ可能である場合に限る）。

In [ ]:
from operator import itemgetter

r = "ABCDEFG"
assert itemgetter(1)(r) == r[1] == "B"
assert itemgetter(slice(2, None))(r) == r[2:] == "CDEFG"

``` python
operator.attrgetter(attr)
operator.attrgetter(*attrs)
```

`operator.attrgetter(attr)` が返す関数オブジェクトは、オブジェクト `obj` に対して、属性名が `attr` の属性を取得する処理を実行できる。たとえば、`f = attrgetter('name')` とした後で、`f(obj)` を呼び出すと `obj.name` を返す。続けて書くと `attrgetter('name')(obj) == obj.name` である。引数を `attr1, attr2, ...` と複数指定する場合は、`(obj.attr1, obj.attr2, ...)` を返すような関数である。また、属性名はドットを含むこともできる。たとえば、`attrgetter('name.first')(obj) == obj.name.first` である。次と等価である:

``` python
def attrgetter(*items):
    if any(not isinstance(item, str) for item in items):
        raise TypeError('attribute name must be a string')
    if len(items) == 1:
        attr = items[0]
        def g(obj):
            return resolve_attr(obj, attr)
    else:
        def g(obj):
            return tuple(resolve_attr(obj, attr) for attr in items)
    return g

def resolve_attr(obj, attr):
    for name in attr.split("."):
        obj = getattr(obj, name)
    return obj
```

In [ ]:
from itertools import groupby
from operator import attrgetter

class Mail:
    def __init__(self, date, address, subject):
        self.date = date
        self.address = address
        self.subject = subject

received_mails = [
    Mail("2022-01-02", "alice@example.com", "subject1"),
    Mail("2022-02-03", "bob@example.com", "subject2"),
    Mail("2022-03-04", "chris@example.com", "subject3"),
    Mail("2022-04-05", "alice@example.com", "subject4"),
    Mail("2022-05-06", "bob@example.com", "subject5"),
]
sorted_received_mails = sorted(received_mails, key=attrgetter("address"))
for address, mails in groupby(sorted_received_mails, attrgetter("address")):
    print(address, ",".join(mail.subject for mail in mails))

alice@example.com subject1,subject4
bob@example.com subject2,subject5
chris@example.com subject3
